In [20]:
import pandas as pd
import numpy as np
import tensorflow as tf
import pickle
from tqdm import tqdm
from pathlib import Path

%load_ext autoreload
%autoreload 2
from base import datahandler, prediction_models, evaluation, classifier, utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# Define the directory paths
challenge_data_dir = Path('dataset/phase_1_v2')
data_dir = challenge_data_dir / "train"
labels_dir = challenge_data_dir / 'train_labels.csv'

split_dataframes = datahandler.load_and_prepare_dataframes(data_dir, labels_dir, dtype=np.float32)

#some_dataframes = {df_k : split_dataframes[df_k] for df_k in list(split_dataframes.keys())[:1000]}
#['1012', '1383', '1385', '1386', '1471', '1473', '1474'],

In [22]:
input_features = ['Eccentricity', 'Semimajor Axis (m)', 'Inclination (deg)', 'RAAN (deg)',
       'Argument of Periapsis (deg)', 'True Anomaly (deg)', 'Latitude (deg)',
       'Longitude (deg)', 'Altitude (m)', 'X (m)', 'Y (m)', 'Z (m)',
       'Vx (m/s)', 'Vy (m/s)', 'Vz (m/s)']

input_features_reduced = ['Eccentricity', 'Semimajor Axis (m)', 'Inclination (deg)', 'RAAN (deg)',
       'Argument of Periapsis (deg)', 'True Anomaly (deg)', 'Latitude (deg)',
       'Longitude (deg)', 'Altitude (m)']

input_features_reduced_further = ['Eccentricity', 'Semimajor Axis (m)', 'Inclination (deg)', 'RAAN (deg)', 'Latitude (deg)', 'Longitude (deg)']
input_features_new = ['Eccentricity', 'Semimajor Axis (m)', 'Inclination (deg)', 'RAAN (deg)', 'Latitude (deg)', 'Longitude (deg)',
                  'Argument of Periapsis (deg)', 'True Anomaly (deg)']
label_features=['EW', 'EW_Type', 'EW_Node', 'EW_Node_Location', 'NS', 'NS_Type', 'NS_Node', 'NS_Node_Location']

ds_gen = datahandler.DatasetGenerator(split_df=split_dataframes,
                                      exclude_objects=[],
                                      non_transform_features=['Eccentricity',
                                                              'Semimajor Axis (m)',
                                                              #'Inclination (deg)',
                                                              #'RAAN (deg)',
                                                              #'Argument of Periapsis (deg)',
                                                              #'True Anomaly (deg)',
                                                              'Latitude (deg)',
                                                              #'Longitude (deg)',
                                                              ],
                                      diff_transform_features=[],
                                      sin_transform_features=['Inclination (deg)',
                                                              'RAAN (deg)',
                                                              'Argument of Periapsis (deg)',
                                                              'True Anomaly (deg)',
                                                              'Longitude (deg)',
                                                              #'Latitude (deg)'
                                                              ],#['Longitude (deg)', 'Argument of Periapsis (deg)', 'RAAN (deg)'],
                                      sin_cos_transform_features=[],
                                      overview_features_mean=[#'Eccentricity',
                                                              #'Semimajor Axis (m)',
                                                              'Inclination (sin)',
                                                              'RAAN (sin)',
                                                              #'Argument of Periapsis (sin)',
                                                              #'Longitude (sin)'
                                                              ],
                                      overview_features_std=['Latitude (deg)',
                                                             #'Argument of Periapsis (sin)'
                                                             ],
                                      add_daytime_feature=False,
                                      add_yeartime_feature=False,
                                      add_linear_timeindex=False,
                                      with_labels=True,
                                      train_val_split=0.8,
                                      input_stride=1,
                                      padding='zero',
                                      unify_value_ranges=True,
                                      scale=True,
                                      per_object_scaling=False,
                                      pad_location_labels=0,
                                      input_history_steps=16,
                                      input_future_steps=128,
                                      input_dtype=np.float32,
                                      sort_inputs=True,
                                      seed=181,
                                      deepcopy=True)


=========================Creating Generator=======================
Seed: 181
nTrain: 1520 nVal: 380 (0.80)
Padding: zero
Horizons: 16-128 @ stride 1
Scaling: True  
Limiting True Anomaly to [0.0, 360.0] and Longitude to [-180.0, 180.0]
Sin-Transforming features: ['Inclination (deg)', 'RAAN (deg)', 'Argument of Periapsis (deg)', 'True Anomaly (deg)', 'Longitude (deg)']
Sin-Cos-Transforming features: []
Final 11 input features: ['Argument of Periapsis (sin)', 'Eccentricity', 'Inclination (sin)', 'Latitude (deg)', 'Longitude (sin)', 'RAAN (sin)', 'Semimajor Axis (m)', 'True Anomaly (sin)'] + overview of ['Inclination (sin)', 'RAAN (sin)'] (mean) and ['Latitude (deg)'] (std)
=========================Finished Generator=======================


In [23]:
utils.set_random_seed(42)

train_combined, val_combined = ds_gen.get_datasets(batch_size=512, 
                                                   label_features=['EW_Type', 'NS_Type'],#, 'NS_Type'],
                                                   with_identifier=False, 
                                                   only_nodes=True, 
                                                   shuffle=True, 
                                                   stride=1)
print(train_combined.element_spec)
# Cardinality is below number of nodes, because sometimes EW and NS are in the same location

In [ ]:
utils.set_random_seed(42)

#dense_model = prediction_models.Dense_NN(val_combined, conv1d_layers=[], dense_layers=[256,128,64], l2_reg=0.001, mixed_dropout=0.15, lr_scheduler=[30000,0.9], seed=0)
#dense_model = prediction_models.CNN(train_combined, conv_layers=[[64,6],[64,3],[64,3]], l2_reg=0.001, mixed_dropout=0.15, lr_scheduler=[20000,0.8], seed=0)
#dense_model = prediction_models.Dense_NN(train_combined, conv1d_layers=[], dense_layers=[16], l2_reg=0.0001, input_dropout=0.1, mixed_dropout=0.5, lr_scheduler=[10000,0.9], seed=0)
#dense_model = prediction_models.LSTM_NN(train_combined, input_dropout=0.0, mixed_dropout=0.25, lstm_layers=[32,16], dense_layers=[32,16], l2_reg=0.0, lr_scheduler=[25000,0.9], seed=1)
#dense_model = prediction_models.Dense_NN(train_combined, conv1d_layers=[], dense_layers=[1], l2_reg=0.00001, mixed_dropout=0.2, lr_scheduler=[70000,0.9], seed=0)

dense_model = prediction_models.Dense_NN(val_combined,
                                         conv1d_layers=[[48,6,1,2,1],[48,3,1,1,1],[48,3,1,1,1]],
                                         convlstm1d_layers=[],#[[16,4],[16,4],[16,4]],
                                         conv2d_layers=[],#[32,(6,3)],[32,(6,3)],[32,(6,3)]],
                                         dense_layers=[64,32],
                                         lstm_layers=[],
                                         l2_reg=0.001,
                                         input_dropout=0.00,
                                         mixed_dropout_dense=0.05,
                                         mixed_dropout_cnn=0.1,
                                         mixed_dropout_lstm=0.0,
                                         mixed_batchnorm=False, # if True, this fucks up inference big time! maybe re-visit?
                                         lr_scheduler=[0.005],#, 250, 0.9],
                                         output_type='classification',
                                         seed=0)

#dense_model._model = create_timeseries_classification_model((65,6))

dense_model.summary()

# temporary fix to allow class weights
# train_combined= train_combined.map(lambda x,y:(x,y[f'EW_Type']))
# val_combined = val_combined.map(lambda x,y:(x,y[f'EW_Type'])) 

# w_0 = 1.05
# w_1 = 1.2
# w_2 = 1.05
# w_3 = 0.65

hist = dense_model.fit(train_combined,
                       val_ds=val_combined,
                       epochs=500,
                       verbose=2,
                       plot_hist=True,
                       save_best_only=True,
                       early_stopping=50,
                       target_metric='val_EW_Type_accuracy',
                       #class_weight={0: w_0, 1: w_1, 2: w_2, 3: w_3},
                       callbacks=[])
dense_model.evaluate(val_combined)
#dense_model.model.save('models/ew_ns_classifier_new.hdf5')

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input (InputLayer)          [(None, 144, 11)]            0         []                            
                                                                                                  
 conv1d_15 (Conv1D)          (None, 134, 48)              3216      ['Input[0][0]']               
                                                                                                  
 activation_25 (Activation)  (None, 134, 48)              0         ['conv1d_15[0][0]']           
                                                                                                  
 dropout_17 (Dropout)        (None, 134, 48)              0         ['activation_25[0][0]']       
                                                                                            

2024-02-24 13:36:40.922246: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-02-24 13:36:41.136489: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.11GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.



Epoch 1: val_EW_Type_accuracy improved from -inf to 0.62575, saving model to best_model.hdf5
11/11 - 6s - loss: 2.6424 - EW_Type_loss: 1.2630 - NS_Type_loss: 1.0511 - EW_Type_accuracy: 0.4757 - NS_Type_accuracy: 0.6558 - val_loss: 1.7406 - val_EW_Type_loss: 0.7862 - val_NS_Type_loss: 0.5942 - val_EW_Type_accuracy: 0.6257 - val_NS_Type_accuracy: 0.7193 - 6s/epoch - 588ms/step
Epoch 2/500

Epoch 2: val_EW_Type_accuracy improved from 0.62575 to 0.67440, saving model to best_model.hdf5
11/11 - 1s - loss: 1.7408 - EW_Type_loss: 0.7951 - NS_Type_loss: 0.5855 - EW_Type_accuracy: 0.6469 - NS_Type_accuracy: 0.7169 - val_loss: 1.6089 - val_EW_Type_loss: 0.7173 - val_NS_Type_loss: 0.5434 - val_EW_Type_accuracy: 0.6744 - val_NS_Type_accuracy: 0.7388 - 694ms/epoch - 63ms/step
Epoch 3/500

Epoch 3: val_EW_Type_accuracy improved from 0.67440 to 0.68338, saving model to best_model.hdf5
11/11 - 1s - loss: 1.5751 - EW_Type_loss: 0.6942 - NS_Type_loss: 0.5492 - EW_Type_accuracy: 0.6802 - NS_Type_accurac

KeyboardInterrupt: 

In [ ]:
hist = dense_model.fit(train_combined, val_ds=val_combined, early_stopping=40, target_metric='val_EW_Type_accuracy', epochs=100, verbose=2, plot_hist=True, callbacks=[])

In [6]:
#dense_model.model.save('submission/models/ew_ns_classifier.hdf5')

In [ ]:
ds_gen.plot_dataset_items(val_combined)

In [15]:
pred_df = classifier.create_prediction_df(ds_gen=ds_gen,
                                model=dense_model,
                                train=False,
                                test=False,
                                model_outputs=['EW_Type', 'NS_Type'],#, 'NS_Type'],
                                object_limit=None,
                                only_nodes=True,
                                confusion_matrix=False,
                                prediction_batches=3,
                                verbose=1)

2/2 [==============================] - 0s 123ms/step


In [16]:
ground_truth_df = pd.read_csv(challenge_data_dir / 'train_labels.csv')#.sort_values(['ObjectID', 'TimeIndex']).reset_index(drop=True)
# TEMPORARY: remove erronous keys from gt
ground_truth_df=ground_truth_df.loc[~ground_truth_df['ObjectID'].isin([1012, 1383, 1385, 1386, 1471, 1473, 1474])].copy()
majority_df = classifier.apply_one_shot_method(preds_df=pred_df, location_df=ground_truth_df, dirs=['EW', 'NS'])
# ground_truth_df=ground_truth_df.loc[ground_truth_df['TimeIndex']==0]
# majority_df=majority_df.loc[majority_df['TimeIndex']==0]
# ground_truth_df=ground_truth_df.loc[ground_truth_df['Direction']=='EW']
# majority_df=majority_df.loc[majority_df['Direction']=='EW']

# 0.95 with 3-layer cnn and 128@2 horizon

# EW: 0.965, 996 36
# NS: 0.967, 782 27

evaluator = evaluation.NodeDetectionEvaluator(ground_truth=ground_truth_df, participant=majority_df)
precision, recall, f2, rmse, total_tp, total_fp, total_fn, total_df = evaluator.score()
print(f'Precision: {precision:.3f}')
print(f'TP: {total_tp} FP: {total_fp}')

Precision: 0.962
TP: 1760 FP: 70


In [20]:
print(dense_model.model.optimizer.learning_rate)

<tf.Variable 'learning_rate:0' shape=() dtype=float32, numpy=0.005>


In [10]:
pickle.dump(ds_gen.scaler, open('submission/models/ew_ns_classifier_scaler_oneshot_cnn.pkl', 'wb'))
dense_model.model.save('submission/models/ew_ns_classifier_oneshot_cnn.hdf5')

#0.12 0.92 0.950 (no dropout, strong overfitting, no ft-transform)

/home/david/miniconda3/envs/splid-gpu/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
